In [126]:
# Data processing modules
import pandas as pd
import numpy as np

# Machine learning algorithms
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm

# Model saving
import pickle

# Data cleaning
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
 
stopwords_eng = stopwords.words('english')

import string

from unidecode import unidecode

from googletrans import Translator
from autocorrect import Speller

# Check inbox in intervals, Google Translate API access request limit
from time import sleep

# from win32com import client

import json

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kenem001\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [129]:
with open('./models/test/model.pkl', 'rb') as f:
    model = pickle.load(f)

with open('./models/test/vec.pkl', 'rb') as f:
    vec = pickle.load(f)

In [131]:
model.get_params()

{'cv': None,
 'error_score': nan,
 'estimator__C': 1.0,
 'estimator__break_ties': False,
 'estimator__cache_size': 200,
 'estimator__class_weight': None,
 'estimator__coef0': 0.0,
 'estimator__decision_function_shape': 'ovr',
 'estimator__degree': 3,
 'estimator__gamma': 'scale',
 'estimator__kernel': 'rbf',
 'estimator__max_iter': -1,
 'estimator__probability': True,
 'estimator__random_state': None,
 'estimator__shrinking': True,
 'estimator__tol': 0.001,
 'estimator__verbose': False,
 'estimator': SVC(probability=True),
 'n_jobs': None,
 'param_grid': {'kernel': ['linear'],
  'gamma': [0.001, 0.01, 0.1],
  'C': [0.1, 1, 10]},
 'pre_dispatch': '2*n_jobs',
 'refit': True,
 'return_train_score': False,
 'scoring': None,
 'verbose': 0}

In [79]:
df_original = pd.read_csv('../../Data/data_generalised_cleaned.csv')

df = df_original
df = df[df['Translated'].notna()]
df = df[df['Rep_Class'].notna()]

In [64]:
df[df['Rep_Class'] == 'Flights']

,Unnamed: 0,Description,Translated,Rep_Class
76,85,"course taking too long to be finished, also wa...",course taking long finished also asked money,Flights
79,88,"TV service - no service for a week , wants com...",service service week wants compensation,Flights
85,94,"paid deposit on car but no delivery yet, wants...",paid deposit car delivery yet wants deposit back,Flights
89,98,paid a sum of money but works were not carried...,paid sum money works not carried,Flights
93,102,"ordered kitchen, no deposit paid, delay in del...",ordered kitchen deposit paid delay delivery wa...,Flights
...,...,...,...,...
21195,23052,FLIGHT DE;LAY DUE TO DIVERSION,flight day ue diversion,Flights
21413,23288,Airline: LUFTHANSA Flight Number LH1277 Bookin...,airline lufthansa flight number lt1277 booking...,Flights
21499,23379,Delayed package and priority mail was paid,delayed package priority mail paid,Flights
21549,23436,wants deposit back,wants deposit back,Flights


In [33]:
freq = []

for i in model.classes_:
    freq.append([df['Rep_Class'][df['Rep_Class'] == i].count(), i])

freq = sorted(freq)
freq.reverse()

for i, j in freq:
    print(i, j)

7686 Quality of Goods and Service
3697 Contract and Sales
2880 Cancelled
2270 Delivery of Goods/Provision of Service
1869 Other Issues
961 Warranty/Statutory Commercial Guarantees
609 Unfair Commercial Practice
587 Delayed
540 Invoicing/Billing
421 Safety Goods & Service
395 Pricing Tariff
138 Denied
114 Redress
28 Boarding
23 Provider Change/Switching
4 Privacy Data Protection


In [13]:
df['Rep_Class']

0                              Contract and Sales
1                              Contract and Sales
2                              Contract and Sales
3                              Contract and Sales
4                              Contract and Sales
                           ...                   
22217                          Contract and Sales
22218    Warranty/Statutory Commercial Guarantees
22219                          Contract and Sales
22220                                   Cancelled
22221                                   Cancelled
Name: Rep_Class, Length: 22222, dtype: object

In [118]:
def clean(input_text):
    '''
    This function cleans the body received from e-mails so that features
    to be processed by the Support Vector Machine are extracted from it 
    more effectively

    via https://www.geeksforgeeks.org/python-efficient-text-data-cleaning/
    '''

    with open('uk_us.txt', 'r') as f:
        uk_us = f.read()

    uk_us = json.loads(uk_us) 

    apos_dict = {"'s": " is", "n't": " not", "'m": " am", "'ll": " will",
                 "'d": " would", "'ve": " have", "'re": " are"}

    ignore_messages = [
        'CAUTION: This email originated from OUTSIDE the Government Email Infrastructure. DO NOT CLICK LINKS or OPEN attachments unless you recognise the sender and know the content is safe.'
    ]

    for i in ignore_messages:
        if i in input_text:
            input_text = re.sub(i, '', input_text)

    # Convert to unicode
    input_text = unidecode(input_text)

    # Remove URLs, hashtags
    input_text = re.sub(r'https?:\/\/.\S+', '', input_text)
    input_text = re.sub(r'#', '', input_text)

    # Contraction replacement
    for key, value in apos_dict.items():
        if key in input_text:
            input_text = input_text.replace(key, value)

    # Split attached words
    input_text = ' '.join([s for s in re.split(
        '([A-Z][a-z]+[^A-Z]*)', input_text) if s])

    # Remove punctuation
    input_text_list = []
    for word in input_text:
        if word not in string.punctuation:
            input_text_list.append(word)

    input_text_list = ''.join(input_text_list).split(' ')
    input_text = ' '.join(input_text_list)

    # Translation + Spell check
    input_text = re.sub(r'(?<![\w\d])kons(?![\w\d])', 'konsumatur', input_text)
    input_text = re.sub(r'(?<![\w\d])cons(?![\w\d])', 'consumer', input_text)

    lang = ''

    try:
        translator = Translator()

        lang = translator.detect(input_text).lang

        input_text = translator.translate(input_text, dest='en').text
        input_text_list = input_text.split(' ')
    except:
        print('(Input not translated)')

    spell = Speller(lang='en')

    for i in range(len(input_text_list)):
        input_text_list[i] = spell(input_text_list[i])

    input_text = ' '.join(input_text_list)

    # Convert to lowercase
    input_text = input_text.lower()

    # Remove stopwords
    exceptions = ['not']
    ignore = ['hi', 'hello', 'dear', 'sir', 'madam', 'ms', 'mr', 'regards']

    input_text_tokens = input_text.split()

    input_text_list = []

    for word in input_text_tokens:
        if ((word not in stopwords_eng) and (word not in ignore)) or (word in exceptions):
            if word in uk_us:
                input_text_list.append(uk_us[word])
            else:
                input_text_list.append(word)

    

    input_text = ' '.join(input_text_list)

    input_text = re.sub(' +', ' ', input_text)

    return input_text, lang


In [68]:
translator = Translator()

input_text = 'tariff'

translator.translate(input_text, dest='en_us').text

'tariff'

In [102]:
blank_key_words = [
    [[],
     'Contract and Sales'],

    [[],
     'Delivery of Goods/Provision of Service'],

     [[],
     'Flights'],

    [[],
     'Invoicing/Billing'],

    [[],
     'Pricing Tariff'],

    [[],
     'Quality of Goods and Service'],

    [[],
     'Unfair Commercial Practice'],

    [[],
     'Warranty/Statutory Commercial Guarantees']
]

key_words = blank_key_words.copy()

for i in key_words:
    i_path = i.copy()
    i_path[1] = re.sub(' ', '-', i_path[1])
    i_path[1] = re.sub('/', '-', i_path[1])
    # print(i[1])

    path = './keywords/' + i_path[1] + '.txt'

    with open(path, 'r') as f:
        output = [re.sub('\n', '', i) for i in f.readlines()]
        for j in range(len(key_words)):
            if key_words[j][1] == i[1]:
                key_words[j][0].extend(output)


In [103]:
def predict(input_text):
    '''
    This function predicts the category of an e-mail based on its body.
    '''

    # Check which words occured most frequently in the body and check them
    # in order of most to least frequent
    freq = []

    for unique in set(input_text.split(' ')):
        freq.append([input_text.split(' ').count(unique), unique])

    freq = sorted(freq)
    freq.reverse()

    multipliers = []

    for i in key_words:
        multipliers.append([1, i[1]])

    for word_freq in freq:
        for category in key_words:
            for key_word in category[0]:
                # If a word in the body matches one in the list of key words,
                # increase the bias towards the category corresponding to that
                # key word
                if word_freq[1] == key_word:
                    for i in range(len(multipliers)):
                        if category[1] == multipliers[i][1]:
                            multipliers[i][0] += .5 * word_freq[0]

    test = vec.transform(np.array([input_text]))

    results = []

    # Return the predicted likelihood that the body corresponds
    # to each category
    for i in range(len(model.predict_proba(test)[0])):
        results.append(
            [np.round(model.predict_proba(test)[0][i], 2), model.classes_[i]])

    for i in results:
        for multiplier in multipliers:
            if multiplier[1] == i[1]:
                i[0] *= multiplier[0]

    results = sorted(results)
    results.reverse()

    # results = results[:4]

    sum = 0

    # Consider the top 4 categories
    for i in results:
        sum += i[0]

    # Normalise probabilities
    for i in results:
        i[0] *= 1 / sum

    results = results[:4]

    print(f'[PROBABILITIES]')
    for i in results:
        print(f'{str(np.round(i[0] * 100, 1)).rjust(6, " ")}% : {i[1]}')

    print()

    # Check for ambiguity in results, if too ambiguous,
    # assign the category 'Other Issues'
    return results


In [104]:
def generate_response(input_text):
    with open('./messages/default.htm', 'r') as f:
        default = f.read()

    '''
    This function generates a response to the received e-mail based on the
    predicted category return by predict()
    '''

    print(f'{input_text}\n')

    cleaned_text, lang = clean(input_text)

    # print(f'[CLEANED]\n{cleaned_text}\n')

    # Categorical message
    results = predict(cleaned_text)

    category = ''

    for i in results[:2]:
        print('i --- ', i)
        file = re.sub(r'[/\s]', '-', i[1])

        if lang == 'mt':
            path = './messages/Maltese/' + file + '.htm'
        else:
            path = './messages/English/' + file + '.htm'

        with open(path, 'r') as f:
            category += i[1] + ' [' + str(int(np.round(i[0] * 100, 0))) + '%]' + f.read() + '\n\n'

    message = '<html>\n\n' + default + '\n' + category + '\n<html>\n'

    # print(f'[OUTPUT]\n{message}\n')

    # lang = 'Maltese' if lang == 'mt' else 'English'
    # print(f'[LANGUAGE]\n{lang}')

    return message


In [125]:
with open('input.txt', 'r') as f:
    print(predict(clean(f.read())[0]))


[PROBABILITIES]
  53.1% : Contract and Sales
  15.7% : Quality of Goods and Service
   8.7% : Flights
   8.7% : Delivery of Goods/Provision of Service

[[0.5314960629921259, 'Contract and Sales'], [0.15748031496062992, 'Quality of Goods and Service'], [0.08661417322834646, 'Flights'], [0.08661417322834646, 'Delivery of Goods/Provision of Service']]


In [128]:
for i, c in enumerate(model.classes_):
    print(f'{i+1}. {c}')


1. Contract and Sales
2. Delivery of Goods/Provision of Service
3. Flights
4. Invoicing/Billing
5. Pricing Tariff
6. Quality of Goods and Service
7. Unfair Commercial Practice
8. Warranty/Statutory Commercial Guarantees
